In [1]:
PROJECT_NAME: str = "web_graphs"
HDFS_NAMENODE: str = "hdfs://namenode:9000"
INPUT_DIR: str = f"{HDFS_NAMENODE}/input/{PROJECT_NAME}"
OUTPUT_DIR: str = f"{HDFS_NAMENODE}/output/{PROJECT_NAME}"

MASTER_URI = "spark://spark-master:7077"

In [2]:
from pyspark.sql import SparkSession


def spark_session() -> SparkSession:
    spark = (
        SparkSession.builder.appName(PROJECT_NAME.capitalize)
        .master(MASTER_URI)
        .config("spark.driver.memory", "4g")
        .config("spark.hadoop.fs.defaultFS", HDFS_NAMENODE)
        .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
        .getOrCreate()
    )

    print(f"Connected to Spark {spark.version}")

    return spark

In [3]:
WEB_DATA_PATH = f"{INPUT_DIR}/web-Google.txt"

In [4]:
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col


def load_web_data(spark: SparkSession) -> DataFrame:
    raw_data = spark.read.text(WEB_DATA_PATH)
    edges_df = (
        raw_data.filter(~col("value").startswith("#"))
        .selectExpr(
            "split(value, '\\t')[0] as FromNodeID", "split(value, '\\t')[1] as ToNodeID"
        )
        .withColumn("FromNodeID", col("FromNodeID"))
        .withColumn("ToNodeID", col("ToNodeID"))
        .cache()
    )

    return edges_df

# Task 1

Given the Google web graph dataset, please output the sorted list of web pages with the number of outlinks, sorted in descending order of the out-degrees.

## Output Format
a sorted list of pages with their out-degrees
Each line contains: <NodeID>, <out-degree> 

In [42]:
%%time

from pyspark.sql.functions import countDistinct

spark = spark_session()

edges_df = load_web_data(spark)

result_df = (
    edges_df.groupBy("FromNodeID")
    .agg(countDistinct("ToNodeID").alias("OutDegree"))
    .orderBy(col("OutDegree").desc())
    .select(["FromNodeID", "OutDegree"])
)
result_df.cache()

result_df.show(n=10)
print(f"Writing results to '{OUTPUT_DIR}/node_out_degree'\n")
result_df.coalesce(1).write.mode("overwrite").csv(
    f"{OUTPUT_DIR}/node_out_degree", header=True, sep=","
)

result_df.unpersist()

spark.stop()

Connected to Spark 3.5.0
+----------+---------+
|FromNodeID|OutDegree|
+----------+---------+
|    506742|      456|
|    203748|      372|
|    305229|      372|
|    768091|      330|
|    808643|      277|
|    412410|      268|
|    600479|      265|
|    376428|      258|
|    156950|      257|
|    885728|      256|
+----------+---------+
only showing top 10 rows

Writing results to 'hdfs://namenode:9000/output/web_graphs/node_out_degree'

CPU times: user 18.6 ms, sys: 7.68 ms, total: 26.3 ms
Wall time: 5.78 s


# Task 2

Please output the inlink distribution of the top linked web pages, sorted in descending order of the in-degrees.

## Output Format
a sorted list of pages with their in-degrees
Each line contains: <NodeID>, <in-degree>

In [43]:
%%time

from pyspark.sql.functions import countDistinct

spark = spark_session()

edges_df = load_web_data(spark)

result_df = (
    edges_df.groupBy("ToNodeID")
    .agg(countDistinct("FromNodeId").alias("InDegree"))
    .orderBy(col("InDegree").desc())
    .select(["ToNodeID", "InDegree"])
)
result_df.cache()

result_df.show(n=10)
print(f"Writing results to '{OUTPUT_DIR}/node_in_degree'\n")
result_df.coalesce(1).write.mode("overwrite").csv(
    f"{OUTPUT_DIR}/node_in_degree", header=True, sep=","
)

result_df.unpersist()

spark.stop()

Connected to Spark 3.5.0
+--------+--------+
|ToNodeID|InDegree|
+--------+--------+
|  537039|    6326|
|  597621|    5354|
|  504140|    5271|
|  751384|    5182|
|   32163|    5097|
|  885605|    4847|
|  163075|    4731|
|  819223|    4620|
|  605856|    4550|
|  828963|    4484|
+--------+--------+
only showing top 10 rows

Writing results to 'hdfs://namenode:9000/output/web_graphs/node_in_degree'

CPU times: user 27.7 ms, sys: 18 ms, total: 45.7 ms
Wall time: 17.1 s


# Task 3

Design an algorithm that maintains the connectivity of two nodes in an efficient way. Given a node v, please output the list of nodes that v points to, and the list of nodes that points to v. 

## Output Format
Given a node v, 
The first line contains a list of nodes that v points to:
<ToNodeID>, …, <ToNodeID>
The second line contains a list of nodes that point to v
<FromNodeID>, …, <FromNodeID>

In [50]:
%%time


from pyspark.sql.functions import collect_list

spark = spark_session()

edges_df = load_web_data(spark)

outlinks_lookup_df = (
    edges_df.groupBy("FromNodeId")
    .agg(collect_list("ToNodeId").alias("Outlinks"))
    .cache()
)

inlinks_lookup_df = (
    edges_df.groupBy("ToNodeId")
    .agg(collect_list("FromNodeId").alias("Inlinks"))
    .cache()
)


def query_connectivity(target_node_id: int) -> tuple[list[str], list[str]]:
    node_outlinks_row = outlinks_lookup_df.filter(
        col("FromNodeId") == target_node_id
    ).collect()
    outlinks_list = list(node_outlinks_row[0]["Outlinks"] if node_outlinks_row else [])

    node_inlinks_row = inlinks_lookup_df.filter(
        col("ToNodeId") == target_node_id
    ).collect()
    inlinks_list = list(node_inlinks_row[0]["Inlinks"] if node_inlinks_row else [])
    return [outlinks_list, inlinks_list]


outlinks, inlinks = query_connectivity(838275)

print("Outlinks: ", outlinks)
print("Inlinks: ", inlinks)

outlinks_lookup_df.unpersist()
inlinks_lookup_df.unpersist()

spark.stop()

Connected to Spark 3.5.0
Outlinks:  ['131005', '176583', '212627', '244023', '317312', '402846', '490982', '523466', '576105', '624317', '819223', '849081', '853324']
Inlinks:  ['40137', '342', '93843', '84268', '274202', '146221', '392665', '276953', '394000', '286180', '894518', '304796', '232245', '356519', '360360', '402846', '462759', '490982', '41840', '584791', '403431', '624317', '415621', '704945', '457069', '823711', '467595', '853324', '480836', '439', '499733', '201658', '608962', '530844', '615873', '2732', '877635', '204537', '42104', '241476', '872636', '340826', '906287', '356361', '42406', '406066', '43979', '617026', '89014', '880829', '107912', '2862', '480865', '480279', '486273', '705538', '778659', '97399', '800238', '61856', '132153', '345685', '283547', '887799', '812215', '874735', '46099', '7649', '73421', '55221', '130807', '352967', '299718', '576105', '411544', '463632', '567891', '9938', '603467', '445463', '741416', '10221', '873135', '58664', '46553', '1

# Task 5

Compute the PageRank of each node in the graph using MapReduce.

## Output Format
The PageRank of each node in steady state
Each line contains: <NodeID>, <pagerank score>

In [5]:
%%time

from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, count, sum, desc, lit

spark = spark_session()


edges_df = load_web_data(spark).cache()

all_nodes_df = (
    edges_df.select(col("FromNodeID").alias("id"))
    .union(edges_df.select(col("ToNodeID").alias("id")))
    .distinct()
    .cache()
)

num_nodes = all_nodes_df.count()
damping_factor = 0.85
num_iterations = 10

pagerank = all_nodes_df.withColumn("Pagerank", lit(1.0 / num_nodes))

out_degree_df = edges_df.groupBy("FromNodeID").agg(count("ToNodeID").alias("OutDegree"))

for i in range(num_iterations):
    contributions = (
        edges_df.join(pagerank, edges_df.FromNodeID == pagerank.id)
        .join(out_degree_df, edges_df.FromNodeID == out_degree_df.FromNodeID)
        .select(col("ToNodeID"), (col("Pagerank") / col("OutDegree")).alias("Contribution"))
    )

    contrib_sum = contributions.groupBy("ToNodeID").agg(sum("Contribution").alias("ContribSum"))

    pagerank = (
        all_nodes_df.join(contrib_sum, all_nodes_df.id == contrib_sum.ToNodeID, "left")
        .select(
            col("id"),
            (
                (lit(1 - damping_factor) / num_nodes) + 
                (lit(damping_factor) * col("ContribSum"))
            ).alias("Pagerank")
        )
        .fillna((1 - damping_factor) / num_nodes, subset=["Pagerank"])
    )
    
    pagerank = pagerank.localCheckpoint() 

pagerank.orderBy(desc("Pagerank")).show(20, truncate=False)

pagerank.coalesce(1).write.mode("overwrite").csv(f"{OUTPUT_DIR}/pagerank", header=False)

spark.stop()

Connected to Spark 3.5.0
+------+---------------------+
|id    |Pagerank             |
+------+---------------------+
|41909 |6.611676319362237E-4 |
|597621|6.610214001229332E-4 |
|163075|6.509212144290871E-4 |
|537039|6.437781370587465E-4 |
|384666|5.629877976155818E-4 |
|504140|5.485468224242569E-4 |
|486980|5.316674861032378E-4 |
|605856|5.206076345801701E-4 |
|558791|5.167020047970761E-4 |
|32163 |5.120609561908314E-4 |
|551829|5.098495101178531E-4 |
|765334|4.844702851964629E-4 |
|751384|4.7908846276145385E-4|
|425770|4.494755132068257E-4 |
|908351|4.433488784698843E-4 |
|173976|4.421624959253195E-4 |
|7314  |4.325030284498375E-4 |
|213432|4.294738905575172E-4 |
|885605|4.24230560971833E-4  |
|691633|4.2139588034612957E-4|
+------+---------------------+
only showing top 20 rows

CPU times: user 163 ms, sys: 75 ms, total: 238 ms
Wall time: 53 s
